In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the IMDb dataset
imdb_dataset = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
import re

def clean_text(text):
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Remove special characters and digits
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply cleaning to the dataset
def clean_dataset(example):
    example["text"] = clean_text(example["text"])
    return example

cleaned_dataset = imdb_dataset.map(clean_dataset)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text
def tokenize_data(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# Tokenize the dataset
tokenized_dataset = cleaned_dataset.map(tokenize_data, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
from torch.utils.data import Dataset
class IMDbDataset(Dataset):
    """PyTorch Dataset for IMDb reviews."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [14]:
# Create PyTorch datasets
print("Preparing PyTorch datasets...")
train_dataset = IMDbDataset(
    encodings={
        "input_ids": tokenized_dataset["train"]["input_ids"],
        "attention_mask": tokenized_dataset["train"]["attention_mask"]
    },
    labels=tokenized_dataset["train"]["label"]
)

test_dataset = IMDbDataset(
    encodings={
        "input_ids": tokenized_dataset["test"]["input_ids"],
        "attention_mask": tokenized_dataset["test"]["attention_mask"]
    },
    labels=tokenized_dataset["test"]["label"]
)

Preparing PyTorch datasets...


In [18]:
import torch  # Add this at the top of the code
from torch.utils.data import Dataset, DataLoader  # Ensure both Dataset and DataLoader are imported

# Step 5: Create DataLoader for Training and Testing
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Step 6: Verify DataLoader Output
print("Verifying data...")
batch = next(iter(train_loader))
print("Batch input_ids shape:", batch["input_ids"].shape)
print("Batch labels shape:", batch["labels"].shape)



Verifying data...
Batch input_ids shape: torch.Size([16, 512])
Batch labels shape: torch.Size([16])


In [19]:
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import get_scheduler
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Step 1: Load Pre-trained BERT Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# Step 2: Define Optimizer and Learning Rate Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define learning rate scheduler
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Step 3: Training Loop
epochs = 3
print("Starting training...")
for epoch in range(epochs):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    total_loss = 0

    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

# Step 4: Evaluation Loop
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

print("Evaluating model...")
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    all_preds.extend(predictions.cpu().numpy())
    all_labels.extend(batch["labels"].cpu().numpy())

# Calculate Accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting training...


Epoch 1: 100%|██████████| 1563/1563 [39:37<00:00,  1.52s/it, loss=0.261]


Epoch 1 completed. Average Loss: 0.3184


Epoch 2: 100%|██████████| 1563/1563 [39:40<00:00,  1.52s/it, loss=0.0269]


Epoch 2 completed. Average Loss: 0.1350


Epoch 3: 100%|██████████| 1563/1563 [39:41<00:00,  1.52s/it, loss=0.0283]


Epoch 3 completed. Average Loss: 0.0416
Evaluating model...
Test Accuracy: 0.9289


In [20]:
# Step 5: Save the Model and Tokenizer
output_dir = "/content/drive/My Drive/Senti/saved_model"

print("Saving model...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model and tokenizer saved to {output_dir}")


Saving model...
Model and tokenizer saved to /content/drive/My Drive/Senti/saved_model


In [22]:
# Load the saved model and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Reload model and tokenizer
model_path = "/content/drive/My Drive/Senti/saved_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Define custom input
text = "The movie was absolutely fantastic! The storyline and acting were amazing."

# Tokenize and preprocess input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Interpret results
logits = outputs.logits
prediction = torch.argmax(logits, dim=-1).item()

# Print sentiment result
sentiment = "Positive" if prediction == 1 else "Negative"
print(f"Sentiment: {sentiment}")


Sentiment: Positive


In [23]:
from sklearn.metrics import classification_report

# Calculate metrics
print("Detailed Evaluation Metrics:")
print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"]))


Detailed Evaluation Metrics:
              precision    recall  f1-score   support

    Negative       0.93      0.92      0.93     12500
    Positive       0.92      0.93      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



In [24]:
from datasets import load_dataset

# Load CoNLL dataset
print("Loading CoNLL dataset...")
dataset = load_dataset("conll2003")

# Display dataset details
print(dataset)


Loading CoNLL dataset...


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [33]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [34]:
# Step 3: Align labels with tokenized inputs
def align_labels_with_tokens(labels, word_ids):
    """
    Align word-level labels to token-level labels, ensuring special tokens are ignored (-100).
    """
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:  # Convert B-XXX to I-XXX for subsequent tokens
                label += 1
            new_labels.append(label)
    return new_labels

In [35]:
def tokenize_and_align_labels(example):
    """
    Tokenize inputs and align POS labels.
    """
    tokenized_inputs = tokenizer(example['tokens'], truncation=True, is_split_into_words=True)
    aligned_labels = align_labels_with_tokens(example['pos_tags'], tokenized_inputs.word_ids())
    tokenized_inputs['labels'] = aligned_labels
    return tokenized_inputs

In [37]:
def tokenize_and_align_labels(examples):
    """
    Tokenize a batch of examples and align POS labels with word IDs.
    """
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True
    )
    all_labels = examples['pos_tags']  # Batch of label lists

    # Align labels for each example in the batch
    aligned_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        new_labels = []
        current_word = None

        for word_id in word_ids:
            if word_id != current_word:
                current_word = word_id
                label = -100 if word_id is None else labels[word_id]
                new_labels.append(label)
            elif word_id is None:
                new_labels.append(-100)
            else:
                label = labels[word_id]
                if label % 2 == 1:  # Convert B-XXX to I-XXX for subsequent tokens
                    label += 1
                new_labels.append(label)

        aligned_labels.append(new_labels)

    tokenized_inputs['labels'] = aligned_labels
    return tokenized_inputs


In [38]:
# Tokenize and align labels
print("Tokenizing and aligning labels...")
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("Tokenization complete. Sample:", tokenized_datasets["train"][0])


Tokenizing and aligning labels...


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Tokenization complete. Sample: {'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 22, 42, 16, 21, 35, 37, 16, 21, 22, 7, -100]}


In [39]:
# Step 5: Data Collator for padding
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


In [40]:
# Step 6: Load model for Token Classification
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=47)  # 47 POS tag IDs
print("Model loaded successfully!")


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


In [42]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [44]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=30e3cd1e672e66a1b10dd3325a99b17b8fa3b0d930acad5f2b34156bd5503851
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [45]:
# Step 7: Define evaluation metrics
import evaluate
import numpy as np

metric = evaluate.load("seqeval")
label_names = dataset['train'].features['pos_tags'].feature.names  # Retrieve POS tag names


In [46]:
def compute_metrics(eval_preds):
    """
    Compute precision, recall, and F1-score using seqeval.
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Decode predictions and labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': all_metrics['overall_precision'],
        'recall': all_metrics['overall_recall'],
        'f1': all_metrics['overall_f1'],
        'accuracy': all_metrics['overall_accuracy']
    }


In [52]:
# Step 8: Set up Training Arguments
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/Senti/pos_tagging_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/My Drive/Senti/logs",
    logging_steps=10,
    load_best_model_at_end=True,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [53]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-53-900432f768a7>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [54]:
# Step 10: Train the Model
print("Training the model...")
trainer.train()

Training the model...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.206400,0.293303,0.911809,0.920283,0.916026,0.929711
2,0.158600,0.255252,0.925837,0.928477,0.927155,0.937467
3,0.105800,0.256228,0.926139,0.931171,0.928648,0.938600


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: : seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: IN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: JJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarn

TrainOutput(global_step=2634, training_loss=0.2049800508295666, metrics={'train_runtime': 289.6707, 'train_samples_per_second': 145.417, 'train_steps_per_second': 9.093, 'total_flos': 525680396562168.0, 'train_loss': 0.2049800508295666, 'epoch': 3.0})

In [55]:
# Step 11: Evaluate on Test Set
print("Evaluating the model on test set...")
results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", results)


Evaluating the model on test set...


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: : seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VBD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWar

Test Results: {'eval_loss': 0.24932028353214264, 'eval_precision': 0.9210211813766623, 'eval_recall': 0.9196414969786219, 'eval_f1': 0.9203308221005425, 'eval_accuracy': 0.9395849419328406, 'eval_runtime': 8.2459, 'eval_samples_per_second': 418.753, 'eval_steps_per_second': 26.195, 'epoch': 3.0}


In [56]:

# Step 12: Save the Model
print("Saving the fine-tuned model...")
model.save_pretrained("/content/drive/My Drive/Senti/fine_tuned_pos_model")
tokenizer.save_pretrained("/content/drive/My Drive/Senti/fine_tuned_pos_model")
print("Model saved successfully!")

Saving the fine-tuned model...
Model saved successfully!


In [77]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Correct model path
model_path = "/content/drive/My Drive/Senti/fine_tuned_pos_model"

# Load model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForTokenClassification.from_pretrained(model_path, use_safetensors=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Define actual POS tag names from the dataset
label_names = [
    '"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN',
    'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS',
    'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN',
    'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'
]

# Input sentence
sentence = ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]

# Tokenize input
inputs = tokenizer(sentence, return_tensors="pt", is_split_into_words=True)
word_ids = inputs.word_ids()  # Retrieve word IDs for token alignment

# Move inputs to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict POS tags
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Decode predictions
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1).squeeze().tolist()

# Align predictions with original words and POS tags
predicted_labels = []
previous_word_id = None

for idx, word_id in enumerate(word_ids):
    if word_id is None:  # Skip special tokens
        continue
    if word_id != previous_word_id:  # First subword only
        label_index = predictions[idx]
        if label_index < len(label_names):  # Ensure index is valid
            predicted_labels.append(label_names[label_index])
        else:
            predicted_labels.append("UNKNOWN")
    previous_word_id = word_id

# Print results
print("POS Tagging Results:")
for word, label in zip(sentence, predicted_labels):
    print(f"{word}: {label}")


POS Tagging Results:
EU: NNP
rejects: VBZ
German: JJ
call: NN
to: TO
boycott: VB
British: JJ
lamb: NN
.: .


In [89]:
import torch
from transformers import (
    AutoModelForTokenClassification,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

# Paths to your models
pos_model_path = "/content/drive/My Drive/Senti/fine_tuned_pos_model"
sentiment_model_path = "/content/drive/My Drive/Senti/saved_model"

# Load the POS tagging model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pos_model = AutoModelForTokenClassification.from_pretrained(pos_model_path, use_safetensors=True)
pos_model.to(device)
pos_tokenizer = AutoTokenizer.from_pretrained(pos_model_path)

# Load the sentiment analysis model and tokenizer
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_path)
sentiment_model.to(device)
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_path)

# Define POS tag names
label_names = [
    '"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', "IN",
    "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNP", "NNPS", "NNS", "NN|SYM", "PDT", "POS",
    "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN",
    "VBP", "VBZ", "WDT", "WP", "WP$", "WRB"
]

# Combined pipeline function
def combined_pipeline(sentence):
    print(f"\nInput Sentence: {sentence}")

    # POS Tagging (without splitting sentence manually)
    inputs = pos_tokenizer(sentence, return_tensors="pt", is_split_into_words=False, add_special_tokens=True)
    tokens = pos_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    word_ids = inputs.word_ids()  # Retrieve word alignment
    inputs = {k: v.to(device) for k, v in inputs.items()}

    pos_model.eval()
    with torch.no_grad():
        outputs = pos_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()

    # Align predictions with tokens
    pos_tags = []
    aligned_tokens = []
    previous_word_id = None

    for token, pred, word_id in zip(tokens, predictions, word_ids):
        if word_id is None or token in ["[CLS]", "[SEP]"]:  # Skip special tokens
            continue
        if word_id != previous_word_id:  # Only take first token of a word
            pos_tags.append(label_names[pred])
            aligned_tokens.append(token)
        previous_word_id = word_id

    # Sentiment Analysis
    sentiment_inputs = sentiment_tokenizer(sentence, return_tensors="pt", truncation=True)
    sentiment_inputs = {k: v.to(device) for k, v in sentiment_inputs.items()}
    sentiment_model.eval()
    with torch.no_grad():
        sentiment_outputs = sentiment_model(**sentiment_inputs)
    sentiment_prediction = torch.argmax(sentiment_outputs.logits, dim=-1).item()
    sentiment_label = "Positive" if sentiment_prediction == 1 else "Negative"

    # Print Results
    print("\nPOS Tagging Results:")
    for word, tag in zip(aligned_tokens, pos_tags):
        print(f"{word}: {tag}")

    print("\nSentiment Analysis Result:")
    print(f"Sentiment: {sentiment_label}")

# Test the combined pipeline
sentence = "The movie was fantastic and the actors did a great job."
combined_pipeline(sentence)



Input Sentence: The movie was fantastic and the actors did a great job.

POS Tagging Results:
The: DT
movie: NN
was: VBD
fantastic: JJ
and: CC
the: DT
actors: NNS
did: VBD
a: DT
great: JJ
job: NN
.: .

Sentiment Analysis Result:
Sentiment: Positive


In [90]:
from huggingface_hub import notebook_login

notebook_login()